# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import DataScienceHelperLibrary as dsh
import pandas as pd
import numpy as np

from sklearn.metrics import f1_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import GridSearchCV

from sqlalchemy import create_engine

In [2]:
# load data from database
origDb = 'MyDisasterResponse.db'
origMessages = 'MyMessage'

engine = create_engine('sqlite:///{}'.format(origDb))

df = pd.read_sql_table(origMessages, engine)

In [3]:
dsh.AnalyseDataframe(df)

-------------------- Dataframe analysis started --------------------
Shape:  (22621, 41)
Number of duplicate rows:  0
-------------------- Analysis of Columns with NaN values --------------------
Columns having all values: 41, 100.00%
index, type: int64                     0.0
id, type: int64                        0.0
message, type: object                  0.0
original, type: object                 0.0
genre, type: object                    0.0
related, type: int64                   0.0
request, type: int64                   0.0
offer, type: int64                     0.0
aid_related, type: int64               0.0
medical_help, type: int64              0.0
medical_products, type: int64          0.0
search_and_rescue, type: int64         0.0
security, type: int64                  0.0
military, type: int64                  0.0
child_alone, type: int64               0.0
water, type: int64                     0.0
food, type: int64                      0.0
shelter, type: int64              

We don't need the original text because all messages are given.

In [35]:
df = dsh.RemoveColumnsByWildcard(df, 'original')

-------------------- Start finding and removing columns matchting to wildcards: ['original'] --------------------
No column names found matchting to wildcards
-------------------- Finished removing columns matchting to wildcards --------------------


In [6]:
dfsub = df[(df.related > 1)]
#(df.related > 1) &
dfsub

,index,id,message,original,genre,related,request,offer,aid_related,medical_help,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
90,90,117,Where can people in Fort Mercredi can find foo...,ki kote mounn fort mercredi ka jwenn dlo ak ma...,direct,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
178,181,219,We are at Lycee Philippe Guerrier at les Cayes...,Nou lokalize nou nan lyce Philippe Guerrier au...,direct,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
372,375,460,Please We need help at the Altidor Center at T...,Aide pou centre altidor thomassin 32 ru celin ...,direct,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
468,471,576,meteo. .. .. We might get rain.,Un front froid se retrouve sur Cuba ce matin. ...,direct,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
535,538,655,"We are right behind the airport, we have no fo...",Nou gen probleme dlo manje nou pa gen kay tout...,direct,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536,539,656,we need a lot of help please. we're in Martiss...,Nou bezwen anpil d Svp nan Matisant sitout mou...,direct,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
727,736,883,need some help please we ar at carrefour perni...,beswen aide svp nan kfou pernier,direct,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
740,749,897,we have not received anything in CANAPE VERT. ...,Nou pa jwenn anyen nan kanapev ditou w sa nou ...,direct,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
762,771,925,Please come and help us we are in pis Aviation...,"Souple, vin ede nou sou pis aviation.",direct,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
768,777,931,"We never got any help, we only kept going with...","u pa janm jwenn anyen se ak ti mwayen pa nou, ...",direct,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
'''
for i in range(dfsub.shape[0]):
    dsh.PrintLine('Row: ' + str(i))
    for j in range(dfsub.shape[1]):
        if dfsub.columns[j] in ['id', 'index']:
            continue
        if dfsub.iloc[i, j] == 0:
            continue
        print(dfsub.columns[j], ' - ', dfsub.iloc[i, j])
'''

"\nfor i in range(dfsub.shape[0]):\n    dsh.PrintLine('Row: ' + str(i))\n    for j in range(dfsub.shape[1]):\n        if dfsub.columns[j] in ['id', 'index']:\n            continue\n        if dfsub.iloc[i, j] == 0:\n            continue\n        print(dfsub.columns[j], ' - ', dfsub.iloc[i, j])\n"

In [8]:
X = dsh.SelectColumnsByWildcard(df, ['message'])

Y = dsh.RemoveColumnsByWildcard(df, ['index', 'id', 'message'])

-------------------- Start finding and removing columns matchting to wildcards: ['related'] --------------------
Columns found to remove:  ['related']
-------------------- Finished removing columns matchting to wildcards --------------------
-------------------- Start finding and keeping columns matchting to wildcards: ['related'] --------------------
-------------------- Finished keeping columns matchting to wildcards: 1 --------------------


### 2. Write a tokenization function to process your text data

In [7]:
def Tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    cleanTokens = []
    for tok in tokens:
        cleanTok = lemmatizer.lemmatize(tok).lower().strip()
        cleanTokens.append(cleanTok)
    
    return cleanTokens
        
        

### 3. Build a machine learning pipeline
- You'll find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:



# pipeline = Pipeline([
#         ('features', FeatureUnion([

#             ('text_pipeline', Pipeline([
#                 ('vect', CountVectorizer(tokenizer=tokenize)),
#                 ('tfidf', TfidfTransformer())
#             ])),

#             ('starting_verb', StartingVerbExtractor())
#         ])),

#         ('clf', RandomForestClassifier())
#     ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = {
        'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
        'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
        'features__text_pipeline__vect__max_features': (None, 5000, 10000),
        'features__text_pipeline__tfidf__use_idf': (True, False),
        'clf__n_estimators': [50, 100, 200],
        'clf__min_samples_split': [2, 3, 4],
        'features__transformer_weights': (
            {'text_pipeline': 1, 'starting_verb': 0.5},
            {'text_pipeline': 0.5, 'starting_verb': 1},
            {'text_pipeline': 0.8, 'starting_verb': 1},
        )
    }

cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.